# Assignment 1
This file covers the solutions to the 4 questions on assignment 1.
1. Generate samples using a given covariance matrix.
2. Create decision boundaries giving class priors.
3. Implement the PCA dimensionality reduction technqiue.
4. Implement the kNN classifier.